In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 🗓️ Quarterly GDP Data and Growth Rates

Most major economies report Gross Domestic Product (GDP) data every three months, or **quarterly**. Analyzing this high-frequency data is crucial for tracking the current state of the economy and identifying turning points in the business cycle.

However, reporting quarterly *changes* requires careful interpretation, especially regarding growth rates.

# 📈 Annualized Growth Rates from Quarterly Data

A key convention when reporting GDP growth is **annualizing** the quarterly growth rate. This answers the question: "If the economy continued growing at this *quarterly* rate for a full year, what would the total *annual* growth be?"

* **Quarterly Growth Rate ($g_q$):** The percentage change from one quarter to the next.
    $$ g_q = \frac{GDP_t - GDP_{t-1}}{GDP_{t-1}} $$

* **Annualized Growth Rate ($g_a$):** Compounding the quarterly growth rate over four quarters.
    $$ 1 + g_a = (1 + g_q)^4 $$
    $$ g_a = (1 + g_q)^4 - 1 $$

**Why Annualize?** It allows for easier comparison with annual historical data and makes quarterly changes appear more significant (or volatile).

**Important Note:** A small quarterly change results in a much larger annualized rate. For example, a 1% quarterly growth ($g_q = 0.01$) corresponds to an annualized growth rate of $(1.01)^4 - 1 \approx 0.0406$ or 4.06%.

**Seasonal Adjustment:** Official quarterly GDP data is almost always **seasonally adjusted**. This statistical process removes predictable seasonal patterns (e.g., higher spending in Q4 due to holidays, lower activity in Q1 due to winter weather) to make underlying trends easier to see. The simulation below uses hypothetical non-seasonally adjusted levels for simplicity but calculates growth rates as is typically done with adjusted data.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def quarterly_gdp_growth_simulator(q1_gdp=5000.0, q2_gdp=5100.0, q3_gdp=5200.0, q4_gdp=5300.0):
    """
    Calculates and visualizes quarterly GDP levels, quarter-over-quarter growth,
    and annualized growth rates.

    Args:
        q1_gdp, q2_gdp, q3_gdp, q4_gdp (float): GDP levels for the four quarters.
    """
    # Store GDP levels
    gdp_levels = np.array([q1_gdp, q2_gdp, q3_gdp, q4_gdp])
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']

    # Calculate Quarter-over-Quarter (QoQ) Growth Rates
    # g_q = (GDP_t / GDP_{t-1}) - 1
    gdp_prev = gdp_levels[:-1]
    gdp_curr = gdp_levels[1:]
    # Handle potential division by zero if previous quarter GDP is zero
    qoq_growth = np.full(4, np.nan) # Initialize with NaN
    valid_idx = gdp_prev > 1e-9
    qoq_growth[1:][valid_idx] = (gdp_curr[valid_idx] / gdp_prev[valid_idx]) - 1.0

    # Calculate Annualized Growth Rates from QoQ rates
    # g_a = (1 + g_q)^4 - 1
    annualized_growth = np.full(4, np.nan)
    valid_idx_gq = ~np.isnan(qoq_growth) & (1 + qoq_growth > 0) # Ensure base is positive for power
    annualized_growth[valid_idx_gq] = ((1 + qoq_growth[valid_idx_gq])**4) - 1.0

    # --- Plotting ---
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot GDP Levels (Bar Chart)
    color_level = 'lightblue'
    bars = ax1.bar(quarters, gdp_levels, alpha=0.7, color=color_level, label='GDP Level', edgecolor='black')
    ax1.set_xlabel("Quarter")
    ax1.set_ylabel("GDP Level ($ Billions, Example)", color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    ax1.set_ylim(bottom=0, top=max(gdp_levels.max() * 1.1, 1)) # Ensure non-zero ylim top
    # Add labels to bars
    for bar in bars:
        yval = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:,.0f}', ha='center', va='bottom', fontsize=9)

    # Create secondary axis for growth rates
    ax2 = ax1.twinx()
    color_ann = 'darkgreen'
    color_qoq = 'darkorange'

    # Plot Annualized Growth Rate (Line Chart)
    line_ann = ax2.plot(quarters, annualized_growth * 100, marker='o', color=color_ann, label='Annualized Growth Rate', linewidth=2.5, markersize=7)
    # Plot Quarter-over-Quarter Growth Rate (Line Chart)
    line_qoq = ax2.plot(quarters, qoq_growth * 100, marker='s', color=color_qoq, label='Quarterly Growth Rate', linewidth=1.5, linestyle='--', markersize=5, alpha=0.8)

    ax2.set_ylabel("Growth Rate (%)", color='black')
    ax2.tick_params(axis='y', labelcolor='black')
    # Format y-axis as percentage
    ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%'))
    # Dynamic ylim for growth rates
    valid_growth = np.concatenate([annualized_growth[~np.isnan(annualized_growth)], qoq_growth[~np.isnan(qoq_growth)]]) * 100
    if len(valid_growth) > 0:
         g_min = min(valid_growth.min(), 0)
         g_max = max(valid_growth.max(), 0)
         g_range = max(g_max - g_min, 1) # Avoid zero range
         ax2.set_ylim(g_min - g_range*0.1 - 1, g_max + g_range*0.1 + 1)
    else:
         ax2.set_ylim(-5, 5) # Default range if no valid growth


    ax1.set_title('Quarterly GDP Levels and Growth Rates')
    ax1.grid(True, axis='y', linestyle='--', alpha=0.5)

    # Combine legends
    lines = bars.patches + line_ann + line_qoq
    labels = [l.get_label() for l in lines]
    # Place legend carefully
    fig.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, 0.03), ncol=3, fontsize='small')

    fig.tight_layout(rect=[0, 0.05, 1, 1]) # Adjust layout to make space for legend
    plt.show()

    # --- Display Growth Table ---
    table_md = "### 📈 Growth Rate Summary:\n\n"
    table_md += "| Quarter | GDP Level | Quarterly Growth (QoQ) | Annualized Growth |\n"
    table_md += "|---|---|---|---|\n"
    for i in range(4):
        qg_str = f"{qoq_growth[i]:.2%}" if not np.isnan(qoq_growth[i]) else "---"
        ag_str = f"{annualized_growth[i]:.2%}" if not np.isnan(annualized_growth[i]) else "---"
        table_md += f"| {quarters[i]} | ${gdp_levels[i]:,.0f} | {qg_str} | {ag_str} |\n"
    display(Markdown(table_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(quarterly_gdp_growth_simulator,
         q1_gdp=FloatSlider(value=5000.0, min=1000, max=10000, step=100, description='Q1 GDP:', style=style, layout=layout, readout_format=',.0f'),
         q2_gdp=FloatSlider(value=5100.0, min=1000, max=10000, step=100, description='Q2 GDP:', style=style, layout=layout, readout_format=',.0f'),
         q3_gdp=FloatSlider(value=5200.0, min=1000, max=10000, step=100, description='Q3 GDP:', style=style, layout=layout, readout_format=',.0f'),
         q4_gdp=FloatSlider(value=5300.0, min=1000, max=10000, step=100, description='Q4 GDP:', style=style, layout=layout, readout_format=',.0f')
        );


interactive(children=(FloatSlider(value=5000.0, description='Q1 GDP:', layout=Layout(width='95%'), max=10000.0…

# 🗓️ Quarterly GDP and Growth Rates

Most countries report GDP **quarterly**.

But there's a twist:
- A 1% increase this quarter → **4.06% annualized** growth!
\[
g_{\text{annual}} = (1 + g_{\text{quarter}})^4 - 1
\]


# 🔄 Seasonal Adjustment

Some quarters always have more spending:
- 🎄 Q4 = holiday boom
- 🏖️ Q2 = summer tourism

We use **seasonal adjustment** to smooth the series.


# 📊 How Growth is Reported

| Quarter | GDP ($B) | Quarterly Growth | Annualized Growth |
|---------|----------|------------------|-------------------|
| Q1      | 5,000    | —                | —                 |
| Q2      | 5,100    | 2.00%            | 8.24%             |
| Q3      | 5,200    | 1.96%            | 8.16%             |

> Watch out: **small quarterly changes** look huge when annualized!